In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
#função que lê os csv de idade e os deixa no formato desejado
def trata_dados(df, ano, pop_name):
    df['ID_Municipio'] = df['Município'].str[:6] #coluna do codigo do municipio
    df['Município'] = df['Município'].str[7:] #coluna do nome do municipio
    df['ano'] = ano #coluna do ano dos dados

    df.columns = ['Município', '0a4', '5a9', '10a14', '15a19', '20a24', '25a29',
              '30a34', '35a39', '40a44', '45a49', '50a54', '55a59', '60a64', 
              '65a69', '70a74', '75a79', '>=80anos', 'Total', 'ID_Municipio', 'ano']
        
    df['ID_Municipio'] = pd.to_numeric(df['ID_Municipio'])
        
    #nao sei explicar direito o que faz, mas deixa tudo no formato desejado
    df = df.melt(id_vars=['Município', 'ID_Municipio', 'ano'], var_name='faixa etaria', value_name=pop_name)
    
    return df

In [1]:
#gera o df com a populacao total padrao de 2010

cols = ['Município', '0a4', '5a9', '10a14', '15a19', '20a24', '25a29',
              '30a34', '35a39', '40a44', '45a49', '50a54', '55a59', '60a64', 
              '65a69', '70a74', '75a79', '>=80anos', 'Total']

df = pd.read_csv("dados populacionais/dados total/2010total.csv", encoding='latin1', sep=';')

padrao_t = df.iloc[[-1]] #pega a ultima coluna, que contém a taxa padrão por faixa etária de todo o estado
padrao_t.columns = cols #enumera as faixas

#melt meio que agrega várias colunas em uma só, e cria novas linhas de acordo. 
#No caso, ele pega as várias colunas de idades e inclui todas elas em uma só coluna "faixa etaria"
#e adiciona uma linha agora pras respectivas faixas etárias de cada município
padrao_t = padrao_t.melt(id_vars=['Município'], var_name='faixa etaria', value_name="pop_pad_total")
padrao_t.drop(labels=['Município'], axis=1, inplace=True)


#gera o df com a populacao masculina padrao de 2010

df = pd.read_csv("dados populacionais/dados homem/homem2010.csv", encoding='latin1', sep=';')

padrao_h = df.iloc[[-1]]
padrao_h.columns = cols

padrao_h = padrao_h.melt(id_vars=['Município'], var_name='faixa etaria', value_name="pop_pad_homem")
padrao_h.drop(labels=['Município'], axis=1, inplace=True)

#gera o df com a populacao feminina padrao de 2010

df = pd.read_csv("dados populacionais/dados mulher/mulher2010.csv", encoding='latin1', sep=';')

padrao_m = df.iloc[[-1]]
padrao_m.columns = cols

padrao_m = padrao_m.melt(id_vars=['Município'], var_name='faixa etaria', value_name="pop_pad_mulher")
padrao_m.drop(labels=['Município'], axis=1, inplace=True)

NameError: name 'pd' is not defined

In [4]:
#populacao total
df = pd.read_csv("dados populacionais/dados total/2010total.csv", encoding='latin1', sep=';')
df.drop(df.tail(1).index, inplace=True) #remove a linha com os dados agregados do estado (indesejada)

df = trata_dados(df, 2010, "pop_total")

#populacao masculina
df_h = pd.read_csv("dados populacionais/dados homem/homem2010.csv", encoding='latin1', sep=';')
df_h.drop(df_h.tail(1).index, inplace=True) #remove a linha com os dados agregados do estado (indesejada)

df_h = trata_dados(df_h, 2010, "pop_homem")

#populacao feminina
df_m = pd.read_csv("dados populacionais/dados mulher/mulher2010.csv", encoding='latin1', sep=';')
df_m.drop(df_m.tail(1).index, inplace=True) #remove a linha com os dados agregados do estado (indesejada)

df_m = trata_dados(df_m, 2010, "pop_mulher")

In [5]:
directory = os.fsencode("./dados populacionais/dados total/") #populacao total

anos = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

i = 0

#cria o dataframe com populacao total
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    
    if filename != '2010total.csv':
        aux = pd.read_csv("./dados populacionais/dados total/" + filename, encoding='latin1', sep=";")
        aux.drop(aux.tail(1).index, inplace=True) #remove a linha com os dados agregados do estado (indesejada)

        aux = trata_dados(aux, anos[i], "pop_total")

        df = df.append(aux)
        
        i = i + 1 #avanca o index de anos

In [6]:
directory = os.fsencode("./dados populacionais/dados homem/") #populacao masculina

anos = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

i = 0

#cria o dataframe com populacao masculina
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    
    if filename != 'homem2010.csv':
        aux = pd.read_csv("./dados populacionais/dados homem/" + filename, encoding='latin1', sep=";")
        aux.drop(aux.tail(1).index, inplace=True) #remove a linha com os dados agregados do estado (indesejada)

        aux = trata_dados(aux, anos[i], "pop_homem")

        df_h = df_h.append(aux)
        
        i = i + 1 #avanca o index de anos

In [7]:
directory = os.fsencode("./dados populacionais/dados mulher/") #populacao feminina

anos = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

i = 0

#cria o dataframe com populacao feminina
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    
    if filename != 'mulher2010.csv':
        aux = pd.read_csv("./dados populacionais/dados mulher/" + filename, encoding='latin1', sep=";")
        aux.drop(aux.tail(1).index, inplace=True) #remove a linha com os dados agregados do estado (indesejada)

        aux = trata_dados(aux, anos[i], "pop_mulher")

        df_m = df_m.append(aux)
        
        i = i + 1 #avanca o index de anos

In [8]:
df = df.merge(padrao_t, how="inner") #junta a coluna com a população total padrão no df
 
#junta com o df com as colunas para homens e sua população padrão
df = df.merge(df_h, how="inner")
df = df.merge(padrao_h, how="inner")

#junta o df com as colunas para mulheres e sua população padrão
df = df.merge(df_m, how="inner")
df = df.merge(padrao_m, how="inner")

df

,Município,ID_Municipio,ano,faixa etaria,pop_total,pop_pad_total,pop_homem,pop_pad_homem,pop_mulher,pop_pad_mulher
0,Abadia dos Dourados,310010,2010,0a4,418,1338553,211,684629,207,653924
1,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924
2,Abre Campo,310030,2010,0a4,914,1338553,464,684629,450,653924
3,Acaiaca,310040,2010,0a4,279,1338553,135,684629,144,653924
4,Açucena,310050,2010,0a4,687,1338553,368,684629,319,653924
5,Água Boa,310060,2010,0a4,1232,1338553,627,684629,605,653924
6,Água Comprida,310070,2010,0a4,107,1338553,58,684629,49,653924
7,Aguanil,310080,2010,0a4,257,1338553,131,684629,126,653924
8,Águas Formosas,310090,2010,0a4,1446,1338553,740,684629,706,653924
9,Águas Vermelhas,310100,2010,0a4,1025,1338553,509,684629,516,653924


In [9]:
sim = pd.read_csv("sim_agravos.csv") #dataframe contendo os dados de óbitos de 2010 até 2019

In [10]:
#obitos de agravos por faixa etaria municipio ano e semana epidemiologica
aux = sim[['CODMUNRES', 'faixaetaria', 'ano', 'semana', 'agravo', 'IDADE']].groupby(['CODMUNRES', 'faixaetaria', 'ano', 'semana', 'agravo'], as_index=False).count()
aux.columns = ['ID_Municipio', 'faixa etaria', 'ano', 'semana', 'agravo', 'obitos_total']

#adiciona os agravos considerados como "subgrupos" na tabela
aux_sub = sim[['CODMUNRES', 'faixaetaria', 'ano', 'semana', 'subgrupo', 'IDADE']].groupby(['CODMUNRES', 'faixaetaria', 'ano', 'semana', 'subgrupo'], as_index=False).count()
aux_sub.columns = ['ID_Municipio', 'faixa etaria', 'ano', 'semana', 'agravo', 'obitos_total']

aux = aux.append(aux_sub, ignore_index=True) #juntas os dois dataframes verticalmente
aux

,ID_Municipio,faixa etaria,ano,semana,agravo,obitos_total
0,310000,0a4,2010,46.0,infecespperi,1
1,310000,15a19,2016,5.0,veneno,1
2,310000,15a19,2016,20.0,veneno,1
3,310000,15a19,2016,42.0,veneno,1
4,310000,15a19,2018,16.0,veneno,1
5,310000,20a24,2014,8.0,tmcusooutras,1
6,310000,20a24,2016,29.0,tmcusooutras,1
7,310000,20a24,2016,41.0,tmcusooutras,1
8,310000,20a24,2018,1.0,tmcusooutras,1
9,310000,20a24,2018,10.0,veneno,1


In [11]:
#Repete os processos da ultima celula mas pra homens e mulheres, e adiociona as colunas respectivas
#É muita coisa e eu diria que está feio, mas com calma dá pra ler

#obitos por homem
sim_h = sim[sim['sexo2'] == 'M']

aux_h = sim_h[['CODMUNRES', 'faixaetaria', 'ano', 'semana', 'agravo', 'IDADE']].groupby(['CODMUNRES', 'faixaetaria', 'ano', 'semana', 'agravo'], as_index=False).count()
aux_h.columns = ['ID_Municipio', 'faixa etaria', 'ano', 'semana', 'agravo', 'obitos_homem']

aux_sub_h = sim_h[['CODMUNRES', 'faixaetaria', 'ano', 'semana', 'subgrupo', 'IDADE']].groupby(['CODMUNRES', 'faixaetaria', 'ano', 'semana', 'subgrupo'], as_index=False).count()
aux_sub_h.columns = ['ID_Municipio', 'faixa etaria', 'ano', 'semana', 'agravo', 'obitos_homem']

aux_h = aux_h.append(aux_sub_h, ignore_index=True)

#obitos por mulher
sim_m = sim[sim['sexo2'] == 'F']

aux_m = sim_m[['CODMUNRES', 'faixaetaria', 'ano', 'semana', 'agravo', 'IDADE']].groupby(['CODMUNRES', 'faixaetaria', 'ano', 'semana', 'agravo'], as_index=False).count()
aux_m.columns = ['ID_Municipio', 'faixa etaria', 'ano', 'semana', 'agravo', 'obitos_mulher']

aux_sub_m = sim_m[['CODMUNRES', 'faixaetaria', 'ano', 'semana', 'subgrupo', 'IDADE']].groupby(['CODMUNRES', 'faixaetaria', 'ano', 'semana', 'subgrupo'], as_index=False).count()
aux_sub_m.columns = ['ID_Municipio', 'faixa etaria', 'ano', 'semana', 'agravo', 'obitos_mulher']

aux_m = aux_m.append(aux_sub_m, ignore_index=True)

In [12]:
#junta as tabelas de obitos de homens e mulheres e total

aux = aux.merge(aux_h, how='left')
aux = aux.merge(aux_m, how='left')

aux['obitos_homem'] = aux['obitos_homem'].fillna(0)
aux['obitos_mulher'] = aux['obitos_mulher'].fillna(0)

aux['obitos_homem'] = aux['obitos_homem'].astype(int)
aux['obitos_mulher'] = aux['obitos_mulher'].astype(int)

In [13]:
#junta a tabela de óbitos com a tabela populacional, e então calcula as taxas desejadas

taxas = df.merge(aux, how='inner')

taxas['taxa_obitos_total'] = (taxas['obitos_total'] / taxas['pop_total']) * 100000
taxas['taxa_obitos_homem'] = (taxas['obitos_homem'] / taxas['pop_homem']) * 100000
taxas['taxa_obitos_mulher'] = (taxas['obitos_mulher'] / taxas['pop_mulher']) * 100000

taxas['obitos_esp_total'] = taxas['pop_pad_total'] * (taxas['taxa_obitos_total'] / 100000)
taxas['obitos_esp_homem'] = taxas['pop_pad_homem'] * (taxas['taxa_obitos_homem'] / 100000)
taxas['obitos_esp_mulher'] = taxas['pop_pad_mulher'] * (taxas['taxa_obitos_mulher'] / 100000)

taxas['taxa_obitos_esp_total'] = (taxas['obitos_esp_total'] / taxas['pop_pad_total']) * 100000
taxas['taxa_obitos_esp_homem'] = (taxas['obitos_esp_homem'] / taxas['pop_pad_homem']) * 100000
taxas['taxa_obitos_esp_mulher'] = (taxas['obitos_esp_mulher'] / taxas['pop_pad_mulher']) * 100000

taxas

,Município,ID_Municipio,ano,faixa etaria,pop_total,pop_pad_total,pop_homem,pop_pad_homem,pop_mulher,pop_pad_mulher,...,obitos_mulher,taxa_obitos_total,taxa_obitos_homem,taxa_obitos_mulher,obitos_esp_total,obitos_esp_homem,obitos_esp_mulher,taxa_obitos_esp_total,taxa_obitos_esp_homem,taxa_obitos_esp_mulher
0,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0,72.621641,142.450142,0.000000,972.079158,975.254986,0.000000,72.621641,142.450142,0.000000
1,Açucena,310050,2010,0a4,687,1338553,368,684629,319,653924,...,0,145.560408,271.739130,0.000000,1948.403202,1860.404891,0.000000,145.560408,271.739130,0.000000
2,Aimorés,310110,2010,0a4,1707,1338553,875,684629,832,653924,...,0,58.582308,114.285714,0.000000,784.155243,782.433143,0.000000,58.582308,114.285714,0.000000
3,Aiuruoca,310120,2010,0a4,326,1338553,172,684629,154,653924,...,0,306.748466,581.395349,0.000000,4105.990798,3980.401163,0.000000,306.748466,581.395349,0.000000
4,Além Paraíba,310150,2010,0a4,2239,1338553,1134,684629,1105,653924,...,1,44.662796,0.000000,90.497738,597.835194,0.000000,591.786425,44.662796,0.000000,90.497738
5,Além Paraíba,310150,2010,0a4,2239,1338553,1134,684629,1105,653924,...,0,44.662796,88.183422,0.000000,597.835194,603.729277,0.000000,44.662796,88.183422,0.000000
6,Alfenas,310160,2010,0a4,4914,1338553,2524,684629,2390,653924,...,1,20.350020,0.000000,41.841004,272.395808,0.000000,273.608368,20.350020,0.000000,41.841004
7,Alfenas,310160,2010,0a4,4914,1338553,2524,684629,2390,653924,...,0,20.350020,39.619651,0.000000,272.395808,271.247623,0.000000,20.350020,39.619651,0.000000
8,Alfenas,310160,2010,0a4,4914,1338553,2524,684629,2390,653924,...,0,20.350020,39.619651,0.000000,272.395808,271.247623,0.000000,20.350020,39.619651,0.000000
9,Almenara,310170,2010,0a4,3011,1338553,1550,684629,1461,653924,...,0,33.211558,64.516129,0.000000,444.554301,441.696129,0.000000,33.211558,64.516129,0.000000


In [14]:
cidades_selecionadas = ["Betim", 'Brumadinho','Curvelo','Esmeraldas','Florestal','Fortuna de Minas',
                        'Igarapé','Juatuba','Maravilhas','Mário Campos','Martinho Campos',
                        'Papagaios','Pará de Minas','Paraopeba','Pequi','Pompéu','São Joaquim de Bicas',
                        'São José da Varginha','Sarzedo']

cidades_limitrofes = ['Abaeté','Araçaí','Belo Horizonte','Belo Vale','Bom Despacho','Bonfim','Cachoeira da Prata',
                      'Caetanópolis','Capim Branco','Conceição do Pará','Contagem','Cordisburgo','Corinto',
                      'Dores do Indaiá','Felixlândia','Ibirité','Igaratinga','Inhaúma','Inimutaba','Itaúna',
                      'Itabirito','Itatiaiuçu','Leandro Ferreira','Mateus Leme','Matozinhos','Moeda',
                      'Morada Nova de Minas','Morro da Garça','Nova Lima','Onça de Pitangui','Paineiras',
                      'Pedro Leopoldo','Pitangui','Presidente Juscelino','Quartel Geral','Ribeirão das Neves',
                      'Rio Manso','Santana de Pirapama','Santo Hipólito','Sete Lagoas']


cidades_prox_limitrofes = ['Araújos','Augusto de Lima','Baldim','Biquinhas','Carmo do Cajuru','Cedro do Abaeté',
                           'Conceição do Mato Dentro','Confins','Congonhas','Congonhas do Norte','Crucilândia',
                           'Estrela do Indaiá','Funilândia','Gouveia','Itaguara','Jaboticatubas','Jeceaba',
                           'Jequitibá','Lagoa Santa','Lassance','Luz','Moema','Monjolos','Nova Serrana',
                           'Ouro Preto','Perdigão','Piedade dos Gerais','Piracema','Prudente de Morais',
                           'Raposos','Rio Acima','Sabará','São Gonçalo do Abaeté','São Gonçalo do Pará',
                           'São Gotardo','São José da Lapa','Santa Bárbara','Santa Luzia','Santana do Riacho',
                           'Santo Antônio do Monte','Serra da Saudade','Tiros','Três Marias','Vespasiano']

#cria uma coluna com um identificador pra cada um dos grupos de cidades a serem analizados

taxas['grupo'] = np.nan

taxas.loc[(taxas['Município'].isin(cidades_selecionadas)), 'grupo'] = 1
taxas.loc[(taxas['Município'].isin(cidades_limitrofes)), 'grupo'] = 2
taxas.loc[(taxas['Município'].isin(cidades_prox_limitrofes)), 'grupo'] = 3

taxas = taxas[~taxas['grupo'].isnull()]

In [15]:
cod_faixas = {
    '0a4': 1,
    '5a9': 2,
    '10a14': 3, 
    '15a19': 4,
    '20a24': 5,
    '25a29': 6,
    '30a34': 7,
    '35a39': 8,
    '40a44': 9,
    '45a49': 10,
    '50a54': 11,
    '55a59': 12,
    '60a64': 13,
    '65a69': 14,
    '70a74': 15,
    '75a79': 16,
    '>=80anos': 17
}

#cria uma coluna com um código pra cada faixa etária, assim é mais fácil de fazer pesquisas especificas
#a coluna faixa_etaria só contem strings, c
#Ex: pegar a população entre 30 a 59 anos seria pegas as linhas com cod faixa entre 7 e 12. 

taxas['cod_faixas'] = np.nan
taxas["cod_faixas"] = taxas["faixa etaria"].apply(lambda x: cod_faixas.get(x))

taxas

/home/adam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/adam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Município,ID_Municipio,ano,faixa etaria,pop_total,pop_pad_total,pop_homem,pop_pad_homem,pop_mulher,pop_pad_mulher,...,taxa_obitos_homem,taxa_obitos_mulher,obitos_esp_total,obitos_esp_homem,obitos_esp_mulher,taxa_obitos_esp_total,taxa_obitos_esp_homem,taxa_obitos_esp_mulher,grupo,cod_faixas
0,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,142.450142,0.000000,972.079158,975.254986,0.000000,72.621641,142.450142,0.000000,2.0,1
30,Belo Horizonte,310620,2010,0a4,139756,1338553,71389,684629,68367,653924,...,1.400776,0.000000,9.577786,9.590119,0.000000,0.715533,1.400776,0.000000,2.0,1
31,Belo Horizonte,310620,2010,0a4,139756,1338553,71389,684629,68367,653924,...,1.400776,0.000000,9.577786,9.590119,0.000000,0.715533,1.400776,0.000000,2.0,1
32,Belo Horizonte,310620,2010,0a4,139756,1338553,71389,684629,68367,653924,...,2.801552,1.462694,28.733357,19.180238,9.564907,2.146598,2.801552,1.462694,2.0,1
33,Belo Horizonte,310620,2010,0a4,139756,1338553,71389,684629,68367,653924,...,1.400776,0.000000,9.577786,9.590119,0.000000,0.715533,1.400776,0.000000,2.0,1
34,Belo Horizonte,310620,2010,0a4,139756,1338553,71389,684629,68367,653924,...,0.000000,1.462694,9.577786,0.000000,9.564907,0.715533,0.000000,1.462694,2.0,1
35,Belo Horizonte,310620,2010,0a4,139756,1338553,71389,684629,68367,653924,...,0.000000,1.462694,9.577786,0.000000,9.564907,0.715533,0.000000,1.462694,2.0,1
36,Belo Horizonte,310620,2010,0a4,139756,1338553,71389,684629,68367,653924,...,0.000000,1.462694,9.577786,0.000000,9.564907,0.715533,0.000000,1.462694,2.0,1
37,Belo Horizonte,310620,2010,0a4,139756,1338553,71389,684629,68367,653924,...,1.400776,0.000000,9.577786,9.590119,0.000000,0.715533,1.400776,0.000000,2.0,1
38,Belo Horizonte,310620,2010,0a4,139756,1338553,71389,684629,68367,653924,...,0.000000,1.462694,9.577786,0.000000,9.564907,0.715533,0.000000,1.462694,2.0,1


In [16]:
#reajusta os nomes das colunas para que fiquem mais padronizados

taxas.columns = ['municipio', 'id_municipio', 'ano', 'faixa_etaria', 'pop_total',
       'pop_pad_total', 'pop_homem', 'pop_pad_homem', 'pop_mulher',
       'pop_pad_mulher', 'semana', 'agravo', 'obitos_total',
       'obitos_homem', 'obitos_mulher', 'taxa_obitos_total',
       'taxa_obitos_homem', 'taxa_obitos_mulher', 'obitos_esp_total',
       'obitos_esp_homem', 'obitos_esp_mulher', 'taxa_obitos_esp_total',
       'taxa_obitos_esp_homem', 'taxa_obitos_esp_mulher', 'grupo',
       'cod_faixas']

taxas

,municipio,id_municipio,ano,faixa_etaria,pop_total,pop_pad_total,pop_homem,pop_pad_homem,pop_mulher,pop_pad_mulher,...,taxa_obitos_homem,taxa_obitos_mulher,obitos_esp_total,obitos_esp_homem,obitos_esp_mulher,taxa_obitos_esp_total,taxa_obitos_esp_homem,taxa_obitos_esp_mulher,grupo,cod_faixas
0,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,142.450142,0.000000,972.079158,975.254986,0.000000,72.621641,142.450142,0.000000,2.0,1
30,Belo Horizonte,310620,2010,0a4,139756,1338553,71389,684629,68367,653924,...,1.400776,0.000000,9.577786,9.590119,0.000000,0.715533,1.400776,0.000000,2.0,1
31,Belo Horizonte,310620,2010,0a4,139756,1338553,71389,684629,68367,653924,...,1.400776,0.000000,9.577786,9.590119,0.000000,0.715533,1.400776,0.000000,2.0,1
32,Belo Horizonte,310620,2010,0a4,139756,1338553,71389,684629,68367,653924,...,2.801552,1.462694,28.733357,19.180238,9.564907,2.146598,2.801552,1.462694,2.0,1
33,Belo Horizonte,310620,2010,0a4,139756,1338553,71389,684629,68367,653924,...,1.400776,0.000000,9.577786,9.590119,0.000000,0.715533,1.400776,0.000000,2.0,1
34,Belo Horizonte,310620,2010,0a4,139756,1338553,71389,684629,68367,653924,...,0.000000,1.462694,9.577786,0.000000,9.564907,0.715533,0.000000,1.462694,2.0,1
35,Belo Horizonte,310620,2010,0a4,139756,1338553,71389,684629,68367,653924,...,0.000000,1.462694,9.577786,0.000000,9.564907,0.715533,0.000000,1.462694,2.0,1
36,Belo Horizonte,310620,2010,0a4,139756,1338553,71389,684629,68367,653924,...,0.000000,1.462694,9.577786,0.000000,9.564907,0.715533,0.000000,1.462694,2.0,1
37,Belo Horizonte,310620,2010,0a4,139756,1338553,71389,684629,68367,653924,...,1.400776,0.000000,9.577786,9.590119,0.000000,0.715533,1.400776,0.000000,2.0,1
38,Belo Horizonte,310620,2010,0a4,139756,1338553,71389,684629,68367,653924,...,0.000000,1.462694,9.577786,0.000000,9.564907,0.715533,0.000000,1.462694,2.0,1


In [17]:
taxas.to_csv('taxa_sim.csv', index=False)